<a href="https://colab.research.google.com/github/Geonho-Park/Hufs/blob/main/%EC%BD%94%EB%A1%9C%EB%82%98%20%EC%9D%B4%ED%9B%84%20%EA%B5%AD%EB%AF%BC%EC%B2%AD%EC%9B%90%20%EB%8D%B0%EC%9D%B4%ED%84%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 국민청원 데이터 가져오기


- 참고:
  - [국민청원 데이터셋 CSV](https://newhiwoong.github.io/%EA%B5%AD%EB%AF%BC%EC%B2%AD%EC%9B%90/%EA%B5%AD%EB%AF%BC%EC%B2%AD%EC%9B%90-%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B)
  - [청와대 국민청원 https://www1.president.go.kr/petitions](https://www1.president.go.kr/petitions)
  - [Using Selenium with Google Colaboratory https://darektidwell.com/using-selenium-with-google-colaboratory/](https://darektidwell.com/using-selenium-with-google-colaboratory/)
  - [https://stackoverflow.com/questions/56829470/selenium-google-colab-error-chromedriver-executable-needs-to-be-in-path](https://stackoverflow.com/questions/56829470/selenium-google-colab-error-chromedriver-executable-needs-to-be-in-path)
 
- 참고링크 중 "국민청원 데이터셋 CSV"의 내용을 토대로 수정하여 작성
- 1건당 1-2초(google colab에서 더 느림, 실 사용은 로컬을 권장함)

## 확인된 에러의 경우 
1. 없는 페이지
2. 관리자 차단 페이지

In [ ]:
import sys

# install chromium, its driver, and selenium
if 'google.colab' in sys.modules:
    !apt-get update
    !apt install chromium-chromedriver
    !cp /usr/lib/chromium-browser/chromedriver /usr/bin
    !pip install selenium

# set options to be headless, ..
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import re

if 'ipykernel' in sys.modules:
    from tqdm import tqdm_notebook as tqdm
else:
    from tqdm import tqdm

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:12 http://security.ubuntu.com/ub

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [ ]:
def get_petition(wd, number):
    base_url='https://www1.president.go.kr/petitions/'
    try:
        wd.get(base_url+str(number))
        element = WebDriverWait(wd, 2).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'petitionsView_title')))
        #bs = BeautifulSoup(wd.page_source, 'html.parser')
        bs = BeautifulSoup(wd.page_source, 'lxml')

        progress = bs.find('div',{'class':'petitionsView_progress'}).get_text().strip()
        title = bs.find('h3',{'class':'petitionsView_title'}).get_text().strip()
        like = bs.find('h2',{'class':'petitionsView_count'}).get_text().strip()
        like = int(''.join(re.findall('\d+', str(like))))
        if progress == '답변완료':
            contents = bs.find_all('div',{'class':'View_write'})[1].get_text().strip()
            answer = bs.find_all('div',{'class':'pr_tk25'})[1].get_text().strip()[4:].strip()
        else:
            contents = bs.find('div',{'class':'View_write'}).get_text().strip()
            answer = ''
        info = bs.find('ul',{'class':'petitionsView_info_list'}).get_text().split('\n')
        category = info[1][4:]
        sday = info[2][4:]
        eday = info[3][4:]
        proponent = info[4][3:].split()[0]
        return {'number':number, 'progress':progress, 'title':title,  'like':like,
                'contents':contents, 'answer':answer, 'category':category,
                'sday':sday, 'eday':eday, 'proponent':proponent}
    except Exception as e:
        print(number, e)
        return {'number':number, 'progress':'error', 'title':'',  'like':0,
                'contents':str(e), 'answer':'', 'category':'',
                'sday':'', 'eday':'', 'proponent':''}

In [ ]:
if __name__ == '__main__' or 'google.colab' in sys.modules:
    wd = webdriver.Chrome('chromedriver',options=options)
    print(get_petition(wd, 300))
    print(get_petition(wd, 582349))
    wd.quit()

{'number': 300, 'progress': '청원종료', 'title': '대기업의 잘못된 인터넷 개인가입자 민원 처리방식과 위약금', 'like': 0, 'contents': '저는 SK인터넷,TV를 가입해서 사용하고 있는 개인 가입자 입니다.8/22일 휴가를 마치고 집으로 오니 인터넷,TV가 안되어서 SK 장애부서인 106번 으로전화하여 수리요청을 했습니다.상담원이 관할지역 기사님을 알아보고는 오늘은 기사님들이 스케줄이 안되어서 장애처리를 내일 해야 된다고 하였습니다.그래서 오늘 아이들 인터넷으로 숙제도해야되고 TV시청도 해야되니 오늘 늦더라도 고쳐달라고 말하니,조금뒤 기술부서 실장님이 전화를 주셔서 어떻게 해도 금일중으로는  기사님을 수배할수 없기 때문에 장애처리를 할수 없고 내일 오전10시즘에 처리 할수 있다고 말했습니다.A/S기사님들은 오전에 그날 A/S처리  스케줄을 다잡고 나가는데 그러면 소비자 들은 장애가 나면 무조건 다음날 A/S처리를 봤아야 되니까? 소비자 잘못이 아니라 SK장비때문에 장애가 났는데 이거는 SK에서 긴급 장애처리 기사를 더고용해서 당일 장애건도 그날A/S처리를 해줘야 되는거 아닙니까.그리고 SK잘못때문에 장애가 났는데 당일 처리를 못해주면 인터넷 해지를 요청하니 수십만원의 위약금을 물어야 된다고 하는데 이것은 잘못된 처방 아닙니까.자기들 장비때문에 장애가나서 사용못하고 그래서 수리요청하니 당일 처리가 힘들고,그러면 해지하려하니 위약금 내라고 하고,완전히 개인가입자는 봉입니까.그리고 처음 인터넷가입시 장애처리 문제에 대해서는 설명도 없었는데 장애가 나고나니 스케줄 잡아서 내일 A/S받든지 아님 나갈 사람이 없어방법이 없다.알아서 해라라는 식으로 답변을 듣기만 해야되나요.장애신고 하는 사람들은 지금 인터넷,TV가 안되어서 전화를 하는 사람들이지 미리 고장날것을 알고 전화해서 예약스케줄을 잡는 사람들이 아닙니다.SK인터넷 장애처리부서는 생각좀 하시고 일합시다.', 'answer': '', 'category': '기타', 's

## 사용예

In [ ]:
import pandas as pd

## 청와대 국민청원: https://www1.president.go.kr/petitions

def get_p(시작번호 = 593216, 수집량 = 8942): # 100개당 1분정도 소요. Colab에서는 최대 1만개씩 수집 추천
    rtn = []
    wd = webdriver.Chrome('chromedriver',options=options)
    for idx, i in enumerate(tqdm(range(시작번호-수집량, 시작번호))):
        tmp = get_petition(wd, i)
        if idx == 0:
            rtn_c = list(tmp.keys())
        rtn.append(list(tmp.values()))
        # 오래 유지될경우 느려지는 경향이 있어 중간에 초기화
        if idx % 200 == 0:
            wd.quit()
            wd = webdriver.Chrome('chromedriver',options=options)
    return (rtn_c, rtn)


c, rtn = get_p()
%time wd = webdriver.Chrome('chromedriver',options=options)
%time rtn.append(list(get_petition(wd, 593681).values()))
%time rtn.append(list(get_petition(wd, 10).values()))
wd.quit()
결과 = pd.DataFrame(rtn, columns=c)
결과

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


584636 Message: 

585060 Message: 

585143 Message: 

585146 Message: 

585774 Message: 

585826 Message: 

585885 Message: 

585921 Message: 

586281 Message: 

586298 Message: 

586311 Message: 

586312 Message: 

586499 Message: 

586521 Message: 

586790 Message: 

586954 Message: 

587001 Message: 

587181 Message: 

587281 Message: 

587994 Message: 

588107 Message: 

588276 Message: 

588317 Message: 

588512 Message: 

588711 Message: 

588757 Message: 

588799 Message: 

588807 Message: 

589440 Message: 

589630 Message: 

589706 Message: 

589775 Message: 

590041 Message: 

590149 Message: 

590180 Message: 

590277 Message: 

590395 Message: 

590416 list index out of range
590472 Message: 

590550 list index out of range
590626 Message: 

590803 Message: 

590822 Message: 

590966 list index out of range
591177 Message: 

591191 Message: 

591369 Message: 

591675 list index out of range
591808 list index out of range
591863 Message: 

591890 Message: 

591934 Message: 


,number,progress,title,like,contents,answer,category,sday,eday,proponent
0,584274,청원종료,서울지방병무청 탈의실에 설치된 CCTV에 대한 진상규명을 요구한다. 또한 인권위의 ...,267,본인은 2019년 8월 경 서울지방병무청 제1검사장 탈의실에서 믿을 수 없는 것을 ...,,인권/성평등,2020-01-02,2020-02-01,twitter
1,584275,청원종료,주식시장 활성화 및 소액(개미)투자자 보호,271,우리 나라 코스피 시총이 미국 애플보다 작다는 설이 돌 정도로 한국의 주식시장은 저...,,경제민주화,2020-01-02,2020-02-01,naver
2,584276,청원종료,교정기관의 민낮,198,억울한 일로 국민청원을 신청합니다.\n\n 저는 **구치소 **교도관입니다.\n\n...,,행정,2020-01-02,2020-02-01,kakao
3,584277,청원종료,미세먼지 저감 대책,170,미세먼지의 심각성은 이제 적극적인 대안을 요구 하고 있습니다.\n우리 일상에서 가장...,,안전/환경,2020-01-02,2020-02-01,naver
4,584278,청원종료,악질세입자 방지를 위한 세입자보호법을 재정해주세요.,2127,저는 우선 아이셋의 부모입니다.\n식구가 많은편이고 아이들이 성장함에 따라 집이 좁...,,교통/건축/국토,2020-01-02,2020-02-01,naver
...,...,...,...,...,...,...,...,...,...,...
8939,593213,청원종료,****전자재료한국의 화학물질관리법 위반등의 처벌을 요청합니다.,1114,"[OLED재료제조 일본계 화학회사 ****전자재료의 환경,노동,성희롱 관련 각종 불...",,안전/환경,2020-10-02,2020-11-01,naver
8940,593214,청원종료,경기도 공공이송지원단 즉각 중단해야 합니다,2192,이재명 지사 지시로 10월부터 경기도에서는 소방공무원(구급대원)중에서 덩치 좋고 ...,,인권/성평등,2020-10-02,2020-11-01,twitter
8941,593215,청원종료,분양 열기를 틈탄 건설사(**** ***)의 과도한 발코니 확장 비용 요구와 HUG...,1313,안녕하세요 대통령님!\n\n저희는 최근 ****에서 분양한 서울 동대문 **** 아...,,교통/건축/국토,2020-10-02,2020-11-01,kakao
8942,593681,청원진행중,택배기사 주 5일(토요휴무) 근무를 도와주세요,5294,존경하는 국민 여러분 !\n\n저는 강산이 두번씩 변할만큼 오랜시간을 택배업계에 근...,,교통/건축/국토,2020-10-29,2020-11-28,naver


In [ ]:
결과.to_csv('국민청원데이터.csv', header='true', encoding='utf-8')

[https://www1.president.go.kr/petitions/10](https://www1.president.go.kr/petitions/10)
- 관리자에 의해 비공개된 청원입니다